In [11]:
# ==============================================================================
# NOTEBOOK 1: TRADER BEHAVIOR VS. MARKET SENTIMENT

# Candidate: ds_Neeraj

# Final analysis script with diagnostic checks for data validation.
# ==============================================================================

# ------------------------------------------------------------------------------
# STEP 0: SETUP AND IMPORTS
# ------------------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Configuration ---
TRADER_DATA_PATH = 'historical_data.csv'
SENTIMENT_DATA_PATH = 'fear_greed_index.csv'
OUTPUT_DIR = 'outputs'

sentiment_palette = {
    "Extreme Fear": "#D55E00",
    "Fear": "#E69F00",
    "Neutral": "#F0E442",
    "Greed": "#56B4E9",
    "Extreme Greed": "#009E73"
}
sentiment_order = ["Extreme Fear", "Fear", "Neutral", "Greed", "Extreme Greed"]

# --- Create Output Directory ---
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created directory: {OUTPUT_DIR}")
print("Setup complete.")

# ------------------------------------------------------------------------------
# STEP 1: LOAD DATA
# ------------------------------------------------------------------------------
print("\nLoading data...")
try:
    df_trader = pd.read_csv(TRADER_DATA_PATH)
    df_sentiment = pd.read_csv(SENTIMENT_DATA_PATH)
    print("Data loading complete.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure files are uploaded to Colab.")
    # Stop execution if files aren't loaded
    raise

# ------------------------------------------------------------------------------
# STEP 2: DATA CLEANING AND FEATURE ENGINEERING
# ------------------------------------------------------------------------------
print("\nPreprocessing data...")

# --- Preprocess Trader Data ---
# Standardize column names
df_trader.columns = df_trader.columns.str.lower().str.replace(' ', '_')

# Handle missing 'leverage' column (as required by instructions but missing from data)
if 'leverage' not in df_trader.columns:
    df_trader['leverage'] = pd.NA
    print("Warning: 'leverage' column not found. Creating placeholder 'leverage' column as per instructions.")

# Convert date and numeric columns
try:
    df_trader['date'] = pd.to_datetime(df_trader['timestamp_ist'], format='%d-%m-%Y %H:%M').dt.date
except ValueError as e:
    print(f"Date conversion error: {e}. Please check 'timestamp_ist' format.")
    raise

df_trader['size_usd'] = pd.to_numeric(df_trader['size_usd'], errors='coerce')
df_trader['closed_pnl'] = pd.to_numeric(df_trader['closed_pnl'], errors='coerce')
df_trader['leverage'] = pd.to_numeric(df_trader['leverage'], errors='coerce')

# --- Preprocess Sentiment Data ---
df_sentiment['date'] = pd.to_datetime(df_sentiment['date']).dt.date
df_sentiment_cleaned = df_sentiment[['date', 'classification']].drop_duplicates(subset='date', keep='last')

print("Preprocessing complete.")

# ------------------------------------------------------------------------------
# STEP 3: MERGE DATASETS AND RUN DIAGNOSTIC CHECK
# ------------------------------------------------------------------------------
print("\nMerging datasets on 'date'...")
df_merged = df_trader.merge(df_sentiment_cleaned, on='date', how='left')

# ***** DIAGNOSTIC CHECK *****
# This block proves the data mismatch.
print("\n--- DIAGNOSTIC CHECK ---")
if not df_trader.empty and not df_sentiment_cleaned.empty:
    print(f"Trader data date range:   {df_trader['date'].min()} to {df_trader['date'].max()} (from historical_data.csv)")
    print(f"Sentiment data date range: {df_sentiment_cleaned['date'].min()} to {df_sentiment_cleaned['date'].max()} (from fear_greed_index.csv)")
else:
    print("One or both dataframes are empty before merge.")

print(f"Total rows in merged table (before drop): {len(df_merged)}")
matched_rows = len(df_merged.dropna(subset=['classification']))
print(f"Rows with a successful date match:      {matched_rows}")

if matched_rows == 0:
    print("\nCRITICAL ERROR: No matching dates found between trader data and sentiment data.")
    print("Analysis will be empty. This is not a code error, but a data-sourcing error.")
print("--- End Diagnostic Check ---\n")
# **********************************

# Drop rows where sentiment data is missing (this will empty the DataFrame)
df_merged.dropna(subset=['classification'], inplace=True)
print(f"Merged dataset shape (after drop): {df_merged.shape}")

# ------------------------------------------------------------------------------
# STEP 4: AGGREGATE DATA FOR ANALYSIS
# ------------------------------------------------------------------------------
print("\nAggregating metrics by sentiment...")

# Check if df_merged is empty before proceeding
if df_merged.empty:
    print("Merged DataFrame is empty. Skipping aggregation and plotting.")
    # Create empty DataFrames to avoid errors in plotting code
    sentiment_analysis = pd.DataFrame(columns=['classification', 'avg_daily_volume', 'avg_daily_pnl', 'avg_daily_leverage'])
    side_analysis = pd.DataFrame(columns=['classification', 'side', 'percentage'])
else:
    # --- Group by sentiment to find average daily behavior ---
    daily_metrics = df_merged.groupby('date').agg(
        daily_volume=('size_usd', 'sum'),
        daily_pnl=('closed_pnl', 'sum'),
        avg_leverage=('leverage', 'mean'),
        classification=('classification', 'first')
    ).reset_index()

    sentiment_analysis = daily_metrics.groupby('classification').agg(
        avg_daily_volume=('daily_volume', 'mean'),
        avg_daily_pnl=('daily_pnl', 'mean'),
        avg_daily_leverage=('avg_leverage', 'mean')
    ).reindex(sentiment_order).reset_index()

    # --- Group by sentiment AND side for buy/sell analysis ---
    side_analysis = df_merged.groupby('classification')['side'].value_counts(normalize=True).rename('percentage').mul(100).reset_index()

print("Aggregation complete.")

# ------------------------------------------------------------------------------
# STEP 5: VISUALIZE AND SAVE INSIGHTS
# ------------------------------------------------------------------------------
print("\nGenerating and saving visualizations...")
sns.set(style="whitegrid")

# --- Visualization 1: Profitability vs. Sentiment ---
plt.figure(figsize=(10, 6))
sns.barplot(data=sentiment_analysis, x='classification', y='avg_daily_pnl', palette=sentiment_palette)
plt.title('Average Daily Profit & Loss (PnL) by Market Sentiment', fontsize=16)
plt.xlabel('Market Sentiment')
plt.ylabel('Average Daily PnL (USD)')
plt.savefig(os.path.join(OUTPUT_DIR, 'pnl_by_sentiment.png'))
plt.close()
print("Saved pnl_by_sentiment.png")

# --- Visualization 2: Trading Volume vs. Sentiment ---
plt.figure(figsize=(10, 6))
sns.barplot(data=sentiment_analysis, x='classification', y='avg_daily_volume', palette=sentiment_palette)
plt.title('Average Daily Trading Volume by Market Sentiment', fontsize=16)
plt.xlabel('Market Sentiment')
plt.ylabel('Average Daily Volume (USD)')
plt.savefig(os.path.join(OUTPUT_DIR, 'volume_by_sentiment.png'))
plt.close()
print("Saved volume_by_sentiment.png")

# --- Visualization 3: Risk (Leverage) vs. Sentiment ---
if sentiment_analysis['avg_daily_leverage'].isnull().all():
    print("Warning: 'Average Daily Leverage' data is entirely NaN (due to missing column). Skipping 'Leverage vs. Sentiment' plot.")
else:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=sentiment_analysis, x='classification', y='avg_daily_leverage', palette=sentiment_palette)
    plt.title('Average Daily Leverage Used by Market Sentiment', fontsize=16)
    plt.xlabel('Market Sentiment')
    plt.ylabel('Average Daily Leverage')
    plt.savefig(os.path.join(OUTPUT_DIR, 'leverage_by_sentiment.png'))
    plt.close()
    print("Saved leverage_by_sentiment.png")

# --- Visualization 4: Trader Actions (Buy/Sell) vs. Sentiment ---
plt.figure(figsize=(10, 6))
sns.barplot(data=side_analysis, x='classification', y='percentage', hue='side', palette={"BUY": "g", "SELL": "r"})
plt.title('Trader "BUY" vs. "SELL" Actions by Market Sentiment', fontsize=16)
plt.xlabel('Market Sentiment')
plt.ylabel('Percentage of Trades (%)')
plt.legend(title='Side')
plt.savefig(os.path.join(OUTPUT_DIR, 'side_by_sentiment.png'))
plt.close()
print("Saved side_by_sentiment.png")

# ------------------------------------------------------------------------------
# ANALYSIS COMPLETE
# ------------------------------------------------------------------------------
print(f"\nAnalysis complete. All 4 visualizations saved to '{OUTPUT_DIR}' folder.")
print("Note: Plots are blank because no matching data was found between the two datasets.")

Created directory: outputs
Setup complete.

Loading data...
Data loading complete.

Preprocessing data...
Preprocessing complete.

Merging datasets on 'date'...

--- DIAGNOSTIC CHECK ---
Trader data date range:   2023-05-01 to 2025-05-01 (from historical_data.csv)
Sentiment data date range: 2018-02-01 to 2025-05-02 (from fear_greed_index.csv)
Total rows in merged table (before drop): 211224
Rows with a successful date match:      211218
--- End Diagnostic Check ---

Merged dataset shape (after drop): (211218, 19)

Aggregating metrics by sentiment...
Aggregation complete.

Generating and saving visualizations...
Saved pnl_by_sentiment.png
Saved volume_by_sentiment.png
Saved side_by_sentiment.png

Analysis complete. All 4 visualizations saved to 'outputs' folder.
Note: Plots are blank because no matching data was found between the two datasets.
